The functions below, together with the last cell make it possible to run the pulling via polymarket API once every "INTERVAL" and do this until it reaches the "MAX_RUNS" (it can be set to None if you want the loop to never stop). In the last cell the code prints all the available markets with respective probabilities, token IDs, question and other data. Then from the second loop it overwrites the full dataset with the new pulled data from polymarket but only printes the markets with changed probabilities. It continues doing this process until it reaches a stop condition (variable "full_market_df" always contains the most recently pulled total data and the print is only for the changed probabilities markets). This should be extended by adding a specific condition to be checked in order to make an automatic trade in polymarket. First, data from the newest pull is checked for the condition (trade could be made on some markets) and then only the updated markets should be tested for the same condition (if passed condition then trade)

In [1]:
import time
import requests
import pandas as pd
from IPython.display import display, clear_output
from datetime import datetime
GAMMA_API = "https://gamma-api.polymarket.com"

INTERVAL = 60        # time interval betwwen each run in seconds
MAX_RUNS = 3 # None        # after how many pulls should it stop

In [2]:
import json

def parse_outcome_prices(raw):
    """
    Convert raw outcome_prices to list of floats [YES, NO].
    Returns None if invalid.
    """
    try:
        # If it's already a list, just use it
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        # If it's a string (JSON), parse it
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        else:
            return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None


In [3]:
import json

def parse_token_ids(raw):
    try:
        # If it's already a list, just use it
        if isinstance(raw, list):
            return [raw[0], raw[1]]
        # If it's a string (JSON), parse it
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [parsed[0], parsed[1]]
        else:
            return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None


In [4]:
max_limit = 200
limit = 50

def fetch_polymarket_markets():
    sports = requests.get(f"{GAMMA_API}/sports").json()

    market_data = {}

    for item in sports:
        series = item.get("series")
        if not series:
            continue
        try:
            series = int(series)
        except (TypeError, ValueError):
        # Skip if series or sport_id is not a number
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "tag_id": 100639,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )

            events = resp.json()
            if not events:
                break

            for event in events:
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)

                    if not question or not raw_prices:
                        continue

                    market_data[question] = {
                        "market_id": market.get("id"),
                        "end_date": market.get("endDate"),
                        "Condition_ID": market.get("conditionId"),
                        #"token_address": market.get("clobTokenIds"),                        
                        "outcome_prices": parsed_prices,
                        "token_address": parsed_tokenIDs
                    }

            page_id += 1

    return market_data


In [5]:
import time
from datetime import datetime, timezone
from IPython.display import clear_output
import pandas as pd
import json

previous_market_data = None
full_market_df = pd.DataFrame()
run_count = 0

try:
    while True:
        run_count += 1
        clear_output(wait=True)
        now = datetime.utcnow().replace(tzinfo=timezone.utc)
        print(f"▶ Run {run_count} started at {now}")

        # --- Pull fresh market data ---
        current_market_data = fetch_polymarket_markets()  # dict {question: {...}}

        # --- Save full snapshot immediately ---
        full_results = []
        for question, data in current_market_data.items():
            raw_prices = data.get("outcome_prices")
            raw_ids = data.get("token_address")
            try:
                if isinstance(raw_prices, str) and isinstance(raw_ids, str):
                    parsed_prices = json.loads(raw_prices)
                    parsed_token_ids = json.loads(raw_ids)
                else:
                    parsed_prices = raw_prices
                    parsed_token_ids = raw_ids
                # Fixed: Use 'or' to skip if prices or token IDs are invalid
                if not isinstance(parsed_prices, list) or len(parsed_prices) < 2 or not isinstance(parsed_token_ids, list) or len(parsed_token_ids) < 2:
                    continue
                current_yes = float(parsed_prices[0])
                current_no  = float(parsed_prices[1])
                yes_token_id = parsed_token_ids[0]
                no_token_id = parsed_token_ids[1]
            except (ValueError, IndexError, TypeError, json.JSONDecodeError):
                continue

            full_results.append({
                "question": question,
                "market_id": data["market_id"],
                "current_yes": current_yes,
                "current_no": current_no,
                "datestart": data["end_date"],
                "Yes_TokenID": yes_token_id,
                "No_TokenID": no_token_id
            })

        # --- Convert to DataFrame for full snapshot ---
        if full_results:
            full_market_df = pd.DataFrame(full_results).set_index("question")
        else:
            full_market_df = pd.DataFrame()

        # --- Determine which markets changed ---
        if previous_market_data is None:
            # First run → evaluate all markets
            markets_to_evaluate = current_market_data
            print(f"📊 First run → full market snapshot ({len(full_market_df)} markets)")
            display(full_market_df)
        else:
            markets_to_evaluate = {}
            for question, new_data in current_market_data.items():
                old_data = previous_market_data.get(question)
                if old_data is None or old_data.get("outcome_prices") != new_data.get("outcome_prices"):
                    markets_to_evaluate[question] = new_data
            print(f"Detected {len(markets_to_evaluate)} markets with changed probabilities")

            # --- Prepare delta display ---
            results = []
            for question, data in markets_to_evaluate.items():
                raw_prices = data.get("outcome_prices")
                raw_ids = data.get("token_address")  # Added: Fetch token IDs for this market
                try:
                    if isinstance(raw_prices, str):
                        parsed_prices = json.loads(raw_prices)
                    else:
                        parsed_prices = raw_prices
                    if not isinstance(parsed_prices, list) or len(parsed_prices) < 2:
                        continue
                    
                    # Added: Parse token IDs for this market (same logic as full snapshot)
                    if isinstance(raw_ids, str):
                        parsed_token_ids = json.loads(raw_ids)
                    else:
                        parsed_token_ids = raw_ids
                    if not isinstance(parsed_token_ids, list) or len(parsed_token_ids) < 2:
                        continue
                    
                    current_yes = float(parsed_prices[0])
                    current_no  = float(parsed_prices[1])
                    yes_token_id = parsed_token_ids[0]  # Now correctly set per market
                    no_token_id = parsed_token_ids[1]   # Now correctly set per market
                except (ValueError, IndexError, TypeError, json.JSONDecodeError):
                    continue

                prev_prices = previous_market_data.get(question, {}).get("outcome_prices")
                if prev_prices:
                    try:
                        if isinstance(prev_prices, str):
                            prev_prices = json.loads(prev_prices)
                        prev_yes = float(prev_prices[0])
                        prev_no  = float(prev_prices[1])
                    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
                        prev_yes, prev_no = None, None
                else:
                    prev_yes, prev_no = None, None

                delta_yes = current_yes - prev_yes if prev_yes is not None else None
                delta_no  = current_no  - prev_no  if prev_no  is not None else None

                results.append({
                    "question": question,
                    "market_id": data["market_id"],
                    "prev_yes": prev_yes,
                    "prev_no": prev_no,
                    "current_yes": current_yes,
                    "current_no": current_no,
                    "delta_yes": delta_yes,
                    "delta_no": delta_no,
                    "datestart": data["end_date"],
                    "Yes_TokenID": yes_token_id,  # Now correctly per-market
                    "No_TokenID": no_token_id     # Now correctly per-market
                })

            # --- Display only changed markets ---
            if results:
                df = pd.DataFrame(results).set_index("question")
                changed_df = df[(df['delta_yes'].notna() & (df['delta_yes'] != 0)) |
                                (df['delta_no'].notna() & (df['delta_no'] != 0))]
                if not changed_df.empty:
                    print(f"📊 Markets with changed probabilities: {len(changed_df)}")
                    display(changed_df)
                else:
                    print("No changes detected since last run")
            else:
                print("No changes detected since last run")

        # --- Update previous_market_data ---
        previous_market_data = current_market_data

        # --- Stop condition ---
        if MAX_RUNS and run_count >= MAX_RUNS:
            print("🛑 Max runs reached → stopping")
            break

        print(f"⏳ Sleeping for {INTERVAL // 60} minutes")
        time.sleep(INTERVAL)

except KeyboardInterrupt:
    print("⌨ Manual stop → loop terminated")

▶ Run 3 started at 2026-01-29 17:28:41.970716+00:00
Detected 149 markets with changed probabilities
📊 Markets with changed probabilities: 149


,market_id,prev_yes,prev_no,current_yes,current_no,delta_yes,delta_no,datestart,Yes_TokenID,No_TokenID
question,,,,,,,,,,
Liverpool FC vs. Newcastle United FC: O/U 4.5,1211604,0.200,0.800,0.195,0.805,-0.005,0.005,2026-01-31T20:00:00Z,7400665153297282503723411086522665363932241844...,6649119683914726622176900270225749710390109966...
Chelsea FC vs. West Ham United FC: O/U 1.5,1211620,0.835,0.165,0.830,0.170,-0.005,0.005,2026-01-31T17:30:00Z,5555366510005336444457127547717615849987947975...,9647670000455395259042101003968624346770279180...
Brighton & Hove Albion FC vs. Everton FC: O/U 1.5,1211635,0.750,0.250,0.755,0.245,0.005,-0.005,2026-01-31T15:00:00Z,8881848647678463728756642866823354670841585152...,7755032546692332039086252633402995423893757998...
Newcastle United FC vs. Brentford FC: O/U 1.5,1263132,0.805,0.195,0.545,0.455,-0.260,0.260,2026-02-07T17:30:00Z,3227297855094965438957467731976967476969128164...,6719406239566635671188987803086375591851105262...
Newcastle United FC vs. Brentford FC: Both Teams to Score,1263139,0.585,0.415,0.535,0.465,-0.050,0.050,2026-02-07T17:30:00Z,4187480064719803366040106252503202307365808086...,7485698933734057587269990085997891471451987165...
...,...,...,...,...,...,...,...,...,...,...
Will FC Famalicão win on 2026-02-09?,1236189,0.460,0.540,0.480,0.520,0.020,-0.020,2026-02-09T18:45:00Z,1029388902307794231672917979417063152158804252...,1140826484256019830991954265609276416732913630...
Will FC Famalicão vs. AVS Futebol end in a draw?,1236190,0.450,0.550,0.470,0.530,0.020,-0.020,2026-02-09T18:45:00Z,8721952751304826423690261190190863834056544302...,1084912300773228886158910447345441604722157151...
Will AVS Futebol win on 2026-02-09?,1236191,0.450,0.550,0.470,0.530,0.020,-0.020,2026-02-09T18:45:00Z,9030616077196116653859304074307727631885901960...,4101804445263553800149445865636753418183478098...


🛑 Max runs reached → stopping


In [6]:
full_market_df

,market_id,current_yes,current_no,datestart,Yes_TokenID,No_TokenID
question,,,,,,
Will Leeds United FC win on 2026-01-31?,1209694,0.145,0.855,2026-01-31T15:00:00Z,1872340455201034574754669925164073372926571358...,1271311228017686411126980356368248743396326949...
Will Arsenal FC win on 2026-01-31?,1209696,0.635,0.365,2026-01-31T15:00:00Z,7993857747689916413503201903935785170910933278...,6010870582192508295234825845254030966872820585...
Will Leeds United FC vs. Arsenal FC end in a draw?,1209695,0.215,0.785,2026-01-31T15:00:00Z,9139647912919324822304177613729374026548020255...,1027890376392496869131639885356965023394516531...
Will Wolverhampton Wanderers FC win on 2026-01-31?,1209697,0.305,0.695,2026-01-31T15:00:00Z,7268141994877572145976050491507583029028628510...,1084838434039561436872433876661284808822418190...
Will AFC Bournemouth win on 2026-01-31?,1209699,0.425,0.575,2026-01-31T15:00:00Z,5424051319577253054141116530542293856768487525...,1741423437875654445102637831359015780128602222...
...,...,...,...,...,...,...
O'Higgins FC vs. CD Concepción: O/U 3.5,1292392,0.500,0.500,2026-02-02T23:30:00Z,4275180896264319661508911876329261745557257361...,8437265593000639542882409431146275487091028958...
O'Higgins FC vs. CD Concepción: O/U 4.5,1292396,0.505,0.495,2026-02-02T23:30:00Z,3700458967369524745630556951335656117738344750...,4327242477273579038060263629069870901066405055...
O'Higgins FC vs. CD Concepción: O/U 1.5,1292387,0.505,0.495,2026-02-02T23:30:00Z,7431612717108942172841677466833922042408545203...,9408108016181757484455518392063953512799629605...
